In [2]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock market symbol'}`


Ticker Symbol NET Full Company Profile Financial Performance In 2022, Cloudflare's revenue was $975.24 million, an increase of 48.57% compared to the previous year's $656.43 million. Losses were -$193.38 million, -25.71% less than in 2021. Financial Statements Analyst Forecast According to 22 analysts, the average rating for NET stock is "Hold." Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. Cloudflare, Inc. - Stock Quote & Chart Stock Quote & Chart Stock Quote NYSE: NET 62.18 Nov 09, 2023 4:00 PM ET Today's Change -2.40 (-3.72%) Today's Volume 3,936,311 Today's Open 65.10 Previous Close 64.58 Intraday High 65.40 Intraday Low 62.02 52 Week High 76.07 52 Week Low 37.56 Pricing delayed by 20 minutes. Last Updated Nov 09, 2023 4:00 PM ET Real-time Cloudflar

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "Here's the financial information for Cloudflare Inc. (NYSE: NET):\n\n1. Company Overview:\n   - Cloudflare, Inc. operates a cloud platform that offers a range of network services to companies around the world. The company is headquartered in San Francisco, California.\n   - Market Capitalization: $23,708,800,000\n   - Revenue (TTM): $1,208,972,000\n   - Gross Profit (TTM): $742,631,000\n   - EBITDA: -$81,727,000\n   - EPS (TTM): -$0.61\n   - Profit Margin: -16.7%\n   - Operating Margin (TTM): -11.7%\n   - Return on Assets (TTM): -4.66%\n   - Return on Equity (TTM): -31.2%\n   - 52 Week High: $76.07\n   - 52 Week Low: $37.56\n\n2. Income Statement (2022):\n   - Total Revenue: $975,241,000\n   - Gross Profit: $742,631,000\n   - Operating Income: -$201,203,000\n   - Net Income: -$193,381,000\n